In [1]:
import sqlite3

Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

In [2]:
conn = sqlite3.connect('sqlite_db_pythonsqlite.db')
c = conn.cursor()

In [7]:
c.execute('SELECT name FROM sqlite_master WHERE type = "table" ;')
table_names = c.fetchall()
print(table_names)

[('Bookings',), ('Facilities',), ('Members',)]


In [12]:
c.execute("PRAGMA table_info('Facilities');")
facilities_fields = c.fetchall()
print(facilities_fields)

c.execute("PRAGMA table_info('Bookings');")
booking_fields = c.fetchall()
print(booking_fields)

c.execute("PRAGMA table_info('Members');")
members_fields = c.fetchall()
print(members_fields)

[(0, 'facid', 'int(1)', 1, "'0'", 1), (1, 'name', 'varchar(15)', 0, 'NULL', 0), (2, 'membercost', 'decimal(2,1)', 0, 'NULL', 0), (3, 'guestcost', 'decimal(3,1)', 0, 'NULL', 0), (4, 'initialoutlay', 'int(5)', 0, 'NULL', 0), (5, 'monthlymaintenance', 'int(4)', 0, 'NULL', 0)]
[(0, 'bookid', 'int(4)', 1, "'0'", 1), (1, 'facid', 'int(1)', 0, 'NULL', 0), (2, 'memid', 'int(2)', 0, 'NULL', 0), (3, 'starttime', 'varchar(19)', 0, 'NULL', 0), (4, 'slots', 'int(2)', 0, 'NULL', 0)]
[(0, 'memid', 'int(2)', 1, "'0'", 1), (1, 'surname', 'varchar(17)', 0, 'NULL', 0), (2, 'firstname', 'varchar(9)', 0, 'NULL', 0), (3, 'address', 'varchar(39)', 0, 'NULL', 0), (4, 'zipcode', 'int(5)', 0, 'NULL', 0), (5, 'telephone', 'varchar(14)', 0, 'NULL', 0), (6, 'recommendedby', 'varchar(2)', 0, 'NULL', 0), (7, 'joindate', 'varchar(19)', 0, 'NULL', 0)]


In [20]:
cte = """WITH member_revenue AS (SELECT f.name, SUM(f.membercost * b.slots) as mem_rev
             FROM Bookings as b
             LEFT JOIN Facilities as f
             ON b.facid = f.facid
             WHERE memid != 0
             GROUP BY b.facid),
             guest_revenue AS (SELECT f.name, SUM(f.guestcost * b.slots) as guest_rev
             FROM Bookings as b
             LEFT JOIN Facilities as f
             ON b.facid = f.facid
             WHERE memid = 0
             GROUP BY b.facid)"""
query = """SELECT member_revenue.name, mem_rev + guest_rev AS total_rev
             FROM member_revenue
             FULL JOIN guest_revenue
             ON member_revenue.name = guest_revenue.name
             WHERE mem_rev + guest_rev < 1000"""
c.execute(cte + query)
print(c.fetchall())
             

[('Table Tennis', 180), ('Snooker Table', 240), ('Pool Table', 270)]


Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

In [24]:
c.execute("""SELECT m.surname, m.firstname, (r.surname || r.firstname) AS recommended_by
         FROM Members as m
         LEFT JOIN Members as r
         ON m.memid = r.recommendedby
         ORDER BY m.surname""")
print(c.fetchall())

[('Bader', 'Florence', 'SarwinRamnaresh'), ('Baker', 'Anne', None), ('Baker', 'Timothy', 'CoplinJoan'), ('Boothe', 'Tim', None), ('Butters', 'Gerald', 'GentingMatthew'), ('Coplin', 'Joan', None), ('Crumpet', 'Erica', None), ('Dare', 'Nancy', None), ('Farrell', 'Jemima', 'BakerTimothy'), ('Farrell', 'Jemima', 'PinkerDavid'), ('Farrell', 'David', None), ('GUEST', 'GUEST', None), ('Genting', 'Matthew', 'RumneyHenrietta'), ('Hunt', 'John', None), ('Jones', 'David', 'JonesDouglas'), ('Jones', 'Douglas', None), ('Joplette', 'Janice', 'DareNancy'), ('Joplette', 'Janice', 'JonesDavid'), ('Mackenzie', 'Anna', None), ('Owen', 'Charles', None), ('Pinker', 'David', None), ('Purview', 'Millicent', 'HuntJohn'), ('Rownam', 'Tim', 'BootheTim'), ('Rumney', 'Henrietta', None), ('Sarwin', 'Ramnaresh', None), ('Smith', 'Darren', 'JopletteJanice'), ('Smith', 'Darren', 'ButtersGerald'), ('Smith', 'Darren', 'OwenCharles'), ('Smith', 'Darren', 'SmithJack'), ('Smith', 'Darren', 'MackenzieAnna'), ('Smith', 'Tra

Q12: Find the facilities with their usage by member, but not guests

In [27]:
c.execute("""SELECT f.name, m.firstname, m.surname, SUM(b.slots)
             FROM Bookings as b
             LEFT JOIN Facilities as f
             ON b.facid = f.facid
             LEFT JOIN Members as m
             ON b.memid = m.memid
             WHERE b.memid != 0
             GROUP BY f.name, m.firstname, m.surname""")
print(c.fetchmany(20))

[('Badminton Court', 'Anna', 'Mackenzie', 96), ('Badminton Court', 'Anne', 'Baker', 30), ('Badminton Court', 'Burton', 'Tracy', 6), ('Badminton Court', 'Charles', 'Owen', 18), ('Badminton Court', 'Darren', 'Smith', 432), ('Badminton Court', 'David', 'Jones', 24), ('Badminton Court', 'David', 'Pinker', 21), ('Badminton Court', 'Douglas', 'Jones', 6), ('Badminton Court', 'Erica', 'Crumpet', 6), ('Badminton Court', 'Florence', 'Bader', 27), ('Badminton Court', 'Gerald', 'Butters', 63), ('Badminton Court', 'Henry', 'Worthington-Smyth', 15), ('Badminton Court', 'Hyacinth', 'Tupperware', 3), ('Badminton Court', 'Jack', 'Smith', 36), ('Badminton Court', 'Jemima', 'Farrell', 21), ('Badminton Court', 'John', 'Hunt', 6), ('Badminton Court', 'Millicent', 'Purview', 6), ('Badminton Court', 'Nancy', 'Dare', 30), ('Badminton Court', 'Ponder', 'Stibbons', 48), ('Badminton Court', 'Ramnaresh', 'Sarwin', 21)]


Q13: Find the facilities usage by month, but not guests

In [35]:
c.execute("""SELECT f.name, substr(b.starttime, 6, 2), SUM(b.slots)
             FROM Bookings as b
             LEFT JOIN Facilities as f
             ON b.facid = f.facid
             WHERE b.memid != 0
             GROUP BY f.name, substr(b.starttime, 6, 2)""")
print(c.fetchall())

[('Badminton Court', '07', 165), ('Badminton Court', '08', 414), ('Badminton Court', '09', 507), ('Massage Room 1', '07', 166), ('Massage Room 1', '08', 316), ('Massage Room 1', '09', 402), ('Massage Room 2', '07', 8), ('Massage Room 2', '08', 18), ('Massage Room 2', '09', 28), ('Pool Table', '07', 110), ('Pool Table', '08', 303), ('Pool Table', '09', 443), ('Snooker Table', '07', 140), ('Snooker Table', '08', 316), ('Snooker Table', '09', 404), ('Squash Court', '07', 50), ('Squash Court', '08', 184), ('Squash Court', '09', 184), ('Table Tennis', '07', 98), ('Table Tennis', '08', 296), ('Table Tennis', '09', 400), ('Tennis Court 1', '07', 201), ('Tennis Court 1', '08', 339), ('Tennis Court 1', '09', 417), ('Tennis Court 2', '07', 123), ('Tennis Court 2', '08', 345), ('Tennis Court 2', '09', 414)]
